# Build a motion model from feature matches

## First, get keypoints

Features and neurons

In [1]:
import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from DLC_for_WBFM.utils.feature_detection.utils_features import *
from DLC_for_WBFM.utils.feature_detection.utils_tracklets import *
from DLC_for_WBFM.utils.feature_detection.utils_detection import *
from DLC_for_WBFM.utils.feature_detection.visualization_tracks import *
from DLC_for_WBFM.utils.video_and_data_conversion.import_video_as_array import *
from DLC_for_WBFM.utils.feature_detection.feature_pipeline import *
from DLC_for_WBFM.utils.feature_detection.utils_affine import *

from tqdm import tqdm

In [2]:
# Get the 3d bigtiff folder
bigtiff_folder = r'D:\More-stabilized-wbfm'

btf_fname_red = r'test2020-10-22_16-15-20_test4-channel-0-pco_camera1\test2020-10-22_16-15-20_test4-channel-0-pco_camera1bigtiff.btf'
btf_fname_red = os.path.join(bigtiff_folder, btf_fname_red)

# Actually import
import_opt = {'num_slices':33, 'alpha':0.15}

dat0_vid = get_single_volume(btf_fname_red, 15, **import_opt)
dat1_vid = get_single_volume(btf_fname_red, 30, **import_opt)

## Segment 2 volumes

In [ ]:
opt = {'num_slices':33, 'alpha':1.0}
# Build point clouds for each plane
all_keypoints_pcs0 = build_point_clouds_for_volume(dat0_vid, **import_opt)
all_icp0 = build_correspondence_icp(all_keypoints_pcs0)

all_keypoints_pcs1 = build_point_clouds_for_volume(dat0_vid, **import_opt)
all_icp1 = build_correspondence_icp(all_keypoints_pcs1)

[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.


In [ ]:
all_neurons = [k.points for k in all_keypoints_pcs0]
all_matches = [m.correspondence_set for m in all_icp0]
clust_df0 = build_tracklets_from_matches(all_neurons, all_matches)

all_neurons = [k.points for k in all_keypoints_pcs1]
all_matches = [m.correspondence_set for m in all_icp1]
clust_df1 = build_tracklets_from_matches(all_neurons, all_matches)

In [ ]:
opt = {'num_slices':33, 'alpha':1.0, 'verbose':1}
neurons0, df0, icp0, pcs0 = detect_neurons_using_ICP(dat0_vid, **opt)
neurons1, df1, icp1, pcs1 = detect_neurons_using_ICP(dat1_vid, **opt)

all_features0, all_features1, kp0, kp1, feature_matches = build_features_and_match_2volumes(dat0_vid,dat1_vid,
                                                                      verbose=1,
                                                                      matches_to_keep=0.8,
                                                                      num_features_per_plane=10000,
                                                                      detect_keypoints=True,
                                                                      kp0=neurons0,
                                                                      kp1=neurons1)

all_matches, f2n, all_conf = match_centroids_using_tree(np.array(neurons0), 
                               np.array(neurons1), 
                               all_features0, 
                               all_features1,
                               radius=8,
                               max_nn=50,
                               min_features_needed=5,
                                   verbose=1,
                                     to_mirror=False)

## Alternate way of getting keypoints: my Frame class

In [182]:
opt = {'start_frame':0,
       'num_frames':3,
      'num_reference_frames':2,
      'start_slice':4}
all_matches, all_other_frames, reference_set = track_via_reference_frames(btf_fname_red, **opt)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:13<00:00, 13.79s/it]


In [183]:
print(reference_set)

                ReferenceFrame:
                Frame index: 0 
                Number of neurons: 202 

                ReferenceFrame:
                Frame index: 1 
                Number of neurons: 187 

                RegisteredReferenceFrames:
                Number of frames: 2 



## An example of a full affine model

In [37]:
pts0_unmatched = reference_set.reference_frames[0].keypoint_locs
pts1_unmatched = reference_set.reference_frames[1].keypoint_locs

feature_matches = reference_set.feature_matches[(0,1)]

In [38]:
# Align the keypoints via matches

pts0 = np.zeros((len(feature_matches), 3), dtype=np.float32)
pts1 = np.zeros((len(feature_matches), 3), dtype=np.float32)
for m, match in enumerate(feature_matches):
    pts0[m, :] = pts0_unmatched[match.queryIdx]
    pts1[m, :] = pts1_unmatched[match.trainIdx]

In [39]:
# h = np.transpose(h)
# Note: neurons0 is a dataframe
# n0_2d = np.array([np.array([n[1], n[2]]) for n in neurons0])

val, h, inliers = cv2.estimateAffine3D(pts0,pts1, confidence=0.99)
pts0_trans = cv2.transform(np.array([pts0]), h)[0]

In [40]:
h

array([[ 9.15033187e-01, -2.96604776e-02, -2.51493568e-03,
         1.07420336e+01],
       [-1.99451570e-01,  8.03839099e-01, -6.27004957e-01,
         2.64056234e+02],
       [-1.46517814e-01,  5.14687677e-01,  7.89790389e-01,
        -1.47872252e+02]])

In [57]:
# Make point clouds
pc0 = o3d.geometry.PointCloud()
pc0.points = o3d.utility.Vector3dVector(pts0)
pc0.paint_uniform_color([0,0,0])

pc0_trans = o3d.geometry.PointCloud()
pc0_trans.points = o3d.utility.Vector3dVector(pts0_trans)
pc0_trans.paint_uniform_color([0.5,0.5,0.5])

pc1 = o3d.geometry.PointCloud()
pc1.points = o3d.utility.Vector3dVector(pts1)
pc1.paint_uniform_color([1,0,0])

o3d.visualization.draw_geometries([pc0,pc0_trans, pc1])

## Next, use the affine model locally

In [135]:
i0, i1 = 0, 1

f0 = reference_set.reference_frames[i0]
f1 = reference_set.reference_frames[i1]
all_feature_matches = reference_set.feature_matches[(i0, i1)]

In [136]:
all_lines = None
all_trans = None

for which_neuron in tqdm(range(len(f0.neuron_locs))):
    success, neuron0_trans = propagate_via_affine_model(which_neuron, f0, f1, all_feature_matches)
#     if not success:
#         continue
    pc0_neuron, pc1_trans, pc1_neuron, line = create_affine_visualizations(which_neuron, f0, f1, neuron0_trans)

    if all_lines is None:
        all_lines = line
        all_trans = pc1_trans
    else:
        all_lines = all_lines + line
        all_trans = all_trans + pc1_trans

100%|████████████████████████████████████████████████████████████████████████████████| 202/202 [00:04<00:00, 41.24it/s]


In [91]:
# propagate_via_affine_model(0, f0, f1, all_feature_matches)

In [92]:
#o3d.visualization.draw_geometries([pc0_neuron, all_trans, pc1_neuron, all_lines])

# Finally, turn this into a set of matches

There are three obvious forms of mismatch:
1. Missing neuron in either v0 or v1
2. Related, multiple v0 neurons near one v1 neuron
3. Neuron pushed to an ambiguous location, not very close to a single v1 neuron

To some extent this just needs better neuron segmentation. However, the features are what they are; theoretically the v0 neurons should be pushed close to a v1 neuron, even if it is missing

Strategy 1: greedy
1. Loop over v1 neurons, taking the closest one

--- FOR NOW DO THIS:

Strategy 1.5: greedy with best of 2
1. Loop over v1 neurons, taking the closest two
2. If the closest one is significantly closer, only keep that one


Strategy 2: bipartite
1. Loop over v0 or v1 neurons, getting distances to ~2 closest neighbors
2. Bipartite matching on resulting graph

In [97]:
# Build tree to query v1 neurons
num_n1, pc1_neuron, tree_n1 = build_neuron_tree(f1.neuron_locs, to_mirror=False)

In [103]:
all_trans, len(f0.neuron_locs)

(PointCloud with 187 points., 187)

In [105]:
# Loop over locations of pushed v0 neurons
all_matches = [] # Without confidence
all_conf = []
nn_opt = { 'radius':10.0, 'max_nn':1}
verbose = 1
dist_ratio = 1.5
conf_func = lambda dist : 1.0 / (dist/10+1.0)
for i, neuron in enumerate(np.array(all_trans.points)):
    [k, two_neighbors, two_dist] = tree_n1.search_hybrid_vector_3d(neuron, **nn_opt)
    if k==0:
#         print(f"No close neuron {i}")
        continue
#     print(i, two_neighbors, two_dist)
#     print(neuron, pc1_neuron.points[two_neighbors[0]])
    
    if k==1:
        dist = two_dist[0]
        i_match = two_neighbors[0]
    else:
        if two_dist[0]/two_dist[1] > dist_ratio:
            dist = two_dist[1]
            i_match = two_neighbors[1]
        elif two_dist[1]/two_dist[0] > dist_ratio:
            dist = two_dist[1]
            i_match = two_neighbors[1]
        else:
            if verbose >= 2:
                print(f"Neuron {i} has two close neighbors")
            continue
    
    if verbose >= 2:
        print(f"Found good match for neuron {i}")
    all_matches.append([i, i_match])
    all_conf.append(conf_func(dist))

In [107]:
to_draw = visualize_tracks(f0.neuron_locs, f1.neuron_locs, all_matches)
to_draw.append(all_lines)


In [108]:
o3d.visualization.draw_geometries(to_draw)

In [128]:
# m = all_matches[3]
m = [4, 26]
n0 = f0.neuron_locs[m[0]]
n1 = f1.neuron_locs[m[1]]
print(n0, n1)

[  4.         371.57849731 233.59176636] [ 14.         356.92935791 346.73193156]


# Full video

In [179]:
# Now, full fv
out = track_neurons_full_video(btf_fname_red,
                             start_frame=0,
                             num_frames=500,
                             num_slices=33,
                             alpha=0.15,
                             neuron_feature_radius=5.0,
                             do_mini_max_projections=False,
                             use_affine_matching=True)

  1%|▋                                                                               | 4/499 [00:48<1:39:54, 12.11s/it]

[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.


  1%|▉                                                                               | 6/499 [01:10<1:33:52, 11.43s/it]

[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.


  2%|█▎                                                                              | 8/499 [01:33<1:33:47, 11.46s/it]

...

 99%|█████████████████████████████████████████████████████████████████████████████▏| 494/499 [1:44:42<01:00, 12.07s/it]

[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.


 99%|█████████████████████████████████████████████████████████████████████████████▎| 495/499 [1:44:54<00:48, 12.08s/it]

[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.


100%|██████████████████████████████████████████████████████████████████████████████| 499/499 [1:45:42<00:00, 12.71s/it]


In [180]:
all_matches, all_conf, all_frames = out
clust_df = build_tracklets_from_classes(all_frames, all_matches, all_conf, verbose=0)

In [181]:
import pickle
fname = 'clust_df_dat_affine_nominimax.pickle'
with open(fname, 'wb') as f:
    pickle.dump(clust_df,f)

In [ ]:
i = 4
print(clust_df.at[i, 'slice_ind'])
print(clust_df.at[i, 'all_ind_local'])
print(clust_df.at[i, 'all_prob'])
clust_df.at[i,'all_xyz']

In [146]:
m = [4, 26]
n0 = all_frames[0].neuron_locs[m[0]]
n1 = all_frames[1].neuron_locs[m[1]]
print(n0, n1)

[  4.         364.75       291.36237793] [  8.         470.08862644 223.30122545]


In [178]:
clust_df

,clust_ind,all_ind_local,all_ind_global,all_xyz,all_prob,slice_ind,extended_this_slice,not_finished
0,0,"[12, 54, 27]","[12, 256, 416]","[[9.5, 440.61496353149414, 227.7713222503662],...","[0.6971265294625079, 0.9757246599386857]","[0, 1, 2]",False,False
1,1,"[46, 16]","[46, 218]","[[6.0, 405.4782952202691, 269.23348659939234],...",[0.8531702536005109],"[0, 1]",False,False
2,2,"[56, 10]","[56, 212]","[[7.0, 372.4322916666667, 301.4625481499566], ...",[0.7437136778305012],"[0, 1]",False,False
3,3,"[59, 19, 6]","[59, 221, 395]","[[7.5, 415.73460388183594, 228.13738505045572]...","[0.8217044091495858, 0.7152378711616665]","[0, 1, 2]",False,False
4,4,"[61, 12]","[61, 214]","[[10.5, 424.26441701253253, 239.15997823079428...",[0.7519919277968684],"[0, 1]",False,False
...,...,...,...,...,...,...,...,...
14483,14483,"[107, 124]","[77923, 78073]","[[22.5, 387.4120819091797, 313.207177734375], ...",[0.7926921801850423],"[498, 499]",True,True
14484,14484,"[116, 117]","[77932, 78066]","[[24.5, 449.2133305867513, 290.6131947835286],...",[0.8805861942045736],"[498, 499]",True,True
14485,14485,"[118, 131]","[77934, 78080]","[[25.0, 399.94436922940343, 255.39791870117188...",[0.8436342044967696],"[498, 499]",True,True
14486,14486,"[121, 114]","[77937, 78063]","[[24.0, 394.79122488839283, 320.8195234026228]...",[0.7801511030629661],"[498, 499]",True,True


# Are the point clouds the same between the full function and this notebook?

In [148]:
# This notebook

pc0_neuron = o3d.geometry.PointCloud()
pc0_neuron.points = o3d.utility.Vector3dVector(f0.neuron_locs)
pc0_neuron.paint_uniform_color([0.5,0.5,0.5])

pc0_vid = o3d.geometry.PointCloud()
pc0_vid.points = o3d.utility.Vector3dVector(all_frames[0].neuron_locs)
pc0_vid.paint_uniform_color([0,0,0])

PointCloud with 202 points.

In [150]:
# o3d.visualization.draw_geometries([pc0_neuron, pc0_vid])

In [168]:
# Visualize matches in the video
k = (2,3)
visualize_tracks(all_frames[k[0]].neuron_locs, all_frames[k[1]].neuron_locs, all_matches[k])

[LineSet with 98 lines.,
 PointCloud with 200 points.,
 PointCloud with 277 points.]

In [ ]:
detect_neurons_using_ICP()